In [4]:
import os
import pandas as pd
import io
import json
import itertools

#print(os.getcwd())
csvfiles = list(filter(lambda fn: fn.startswith("Alchemy") and fn.endswith(".csv"),os.listdir()))
#print(csvfiles)

dfStd = pd.read_csv("Alchemy Ingredients - Standard.csv",encoding="utf-8")
dfCC = pd.read_csv("Alchemy Ingredients - Creation Club.csv",encoding="utf-8")

#print(dfStd.head())
#print(dfCC.head())

effects = {}
with io.open("effects.json",mode="r",encoding="utf-8") as effectsJson:
    effects = json.load(effectsJson)

maxWidth = 0
for key,values in effects.items():
    for v in values:
        print(f"{key}: {v}")
        maxWidth = max(maxWidth,len(v))
print()

categories = []
categories.append(dfStd)
categories.append(dfCC)

ingredients = pd.concat(categories,ignore_index=True,sort=False)
categories = None
#print(ingredients.head())
#print(ingredients.shape)

def validateRecipes(recipes,effectsWanted):
    #Remove duplicate combinations from chains and incomplete chains
    
    validRecipes = []
    for recipe in recipes:
            
        trecipe = list(map(lambda i: {"ID": i["ID"], "Name": i["Name"], "Effects": [i["Effect1"],i["Effect2"],i["Effect3"],i["Effect4"]], "Description":i["Description"]},recipe))
        trecipe.sort(key=lambda i: i["Name"])
        
        if not(trecipe in validRecipes):
            iscomplete = True

            effectsAvailiable = []
            for ingredient in trecipe:
                for effect in ingredient["Effects"]:
                    if effect not in effectsAvailiable:
                        effectsAvailiable.append(effect)
            effectsAvailiable = sorted(effectsAvailiable)

            for effectWanted in effectsWanted:
                if not(effectWanted in effectsAvailiable):
                    iscomplete = False
                    break
            if iscomplete:
                if len(trecipe) > 1:
                    validRecipes.append(trecipe)
                else:
                    print("Invalid Recipe")
                    print(effectsWanted)
                    print(trecipe)
    
    return validRecipes            
    
def listIngredients(effectsWanted):
    
    if not(isinstance(effectsWanted,list)):
        raise TypeError("effectsWanted must be a list")
    lenE = len(effectsWanted)
    if lenE < 1:
        raise ValueError("effectsWanted must have at least 1 item")
    if lenE > 4:
        raise ValueError("effectsWanted supports at most 4 items")
    
    tdict = {}
    for effect in effectsWanted:
        tdict[effect] = []
        
    uniqueEffects = tdict.keys()
    
    effectPermutations = list(itertools.permutations(uniqueEffects))

    result = {}    
    for permIndex in range(0,len(effectPermutations)):

        potionEffects = effectPermutations[permIndex]
        
        if permIndex == 0:
            result["Effects"] = list(potionEffects)
            result["Recipes"] = []
        
        if lenE < 2:
            recipe = []
            for index,ingredient in ingredients.iterrows():
                if potionEffects[0] in ingredient.iloc[2:6].values:
                    recipe.append(ingredient)
            recipes = [recipe]
            validRecipes = validateRecipes(recipes,effectsWanted)
            result["Recipes"] = validRecipes
        else:
            chains = []
            for index in range(0,lenE):
                effect1 = potionEffects[-1] if index == 0 else potionEffects[index -1]
                effect2 = potionEffects[index]
                
                tchains = []
                ingCount = 0
                for ingIndex,ingredient in ingredients.iterrows():
                    if (effect1 in ingredient.iloc[2:6].values and effect2 in ingredient.iloc[2:6].values):
                        ingCount += 1
                        if index == 0:
                            tchains.append([ingredient])
                        else:
                            for chainIndex in range(0,len(chains)):
                                
                                chain = chains[chainIndex]
                                
                                tchain = list(chain)
                                if all(list(map(lambda i: not(i.equals(ingredient)),chain))):
                                    tchain.append(ingredient)
                                    tchains.append(tchain)

                chains = list(tchains)
            
            #Remove duplicate combinations from chains and incomplete chains
            validRecipes = validateRecipes(chains,effectsWanted)
            if len(result["Recipes"]) == 0:
                result["Recipes"] = validRecipes
            else:
                for recipe in validRecipes:
                    if recipe not in result["Recipes"]:
                        result["Recipes"].append(recipe)

    return result

potions = {}

# Single effect potions
for k in effects.keys():
    suffix = 'Poison'
    if k in ['Cure','Health','Magicka','Stamina','Resist','Others']:
        suffix = 'Potion'

    effectlist = effects[k]
    for effect in effectlist:
        pname = f'{effect.title()} {suffix}'
        potions[pname] = listIngredients([effect])

# Multiple effect potions
potionGroups = [['Cure'],['Health'],['Magicka'],['Stamina']]
for potionGroup in potionGroups:
    effectsList = []
    for k in potionGroup:
        effectsList = effectsList + effects[k]
        
    effectsCount = len(effectsList)
    effectsCombinations = itertools.combinations(effectsList,min(4,effectsCount))
    for effectsCombination in effectsCombinations:
        potionEffects = [effect.title() for effect in effectsCombination]
        pname = f'{" | ".join(potionEffects)} Potion'
        potions[pname] = listIngredients(list(effectsCombination))

potions["Health Elixir"] = listIngredients(["restore health","fortify health"])
potions["Magicka Elixir"] = listIngredients(["restore magicka","fortify magicka"])
potions["Stamina Elixir"] = listIngredients(["restore stamina","fortify stamina"])
potions["Resist Fire (2)"] = listIngredients(["resist fire","regenerate health"])
potions["Resist Frost (2)"] = listIngredients(["resist frost","restore stamina"])
#potions["Resist Shock (2) - None found"] = listIngredients(["resist shock","restore magicka"])
potions["Cure All (3)"] = listIngredients(["cure disease","cure poison","restore health"])
potions["Fire Remedy (3)"] = listIngredients(["restore health","regenerate health","resist fire"])
#potions["Shock Remedy (3) - None found"] = listIngredients(["restore magicka","regenerate magicka","resist shock"])
potions["Frost Remedy (3)"] = listIngredients(["restore stamina","regenerate stamina","resist frost"])
potions["Elixir of the Thief"] = listIngredients(["fortify sneak","fortify pickpocket","fortify lockpicking"])
potions["Poison (FPS)"] = listIngredients(["fear","paralysis","slow"])

with open("recipes.json",mode="w",encoding="utf-8") as f:
    json.dump(potions,f,indent=4)


Poison: damage health
Poison: damage magicka
Poison: damage magicka regen
Poison: damage stamina
Poison: damage stamina regen
Poison: fear
Poison: frenzy
Poison: lingering damage health
Poison: lingering damage magicka
Poison: lingering damage stamina
Poison: paralysis
Poison: ravage health
Poison: ravage magicka
Poison: ravage stamina
Poison: slow
Poison: weakness to fire
Poison: weakness to frost
Poison: weakness to magic
Poison: weakness to poison
Poison: weakness to shock
Cure: cure disease
Cure: cure poison
Health: fortify health
Health: regenerate health
Health: restore health
Magicka: fortify magicka
Magicka: regenerate magicka
Magicka: restore magicka
Stamina: fortify stamina
Stamina: regenerate stamina
Stamina: restore stamina
Resist: resist fire
Resist: resist frost
Resist: resist magic
Resist: resist poison
Resist: resist shock
Others: fortify alteration
Others: fortify barter
Others: fortify block
Others: fortify carry weight
Others: fortify conjuration
Others: fortify dest